In [116]:
import numpy as np
import matplotlib.pyplot as plt 
import wave
from scipy.io import wavfile

In [117]:
#Découper le fichier en trames de 20 ms

samplerate, data = wavfile.read('sound.wav')
Fe=16000
#durée du signal sonore en secondes
T=len(data)/Fe
#Nombre d'échantillons dans chaque trame
n=int(20*10**(-3)*Fe)
#Découpage en trames
y,i,l=[],0,0
data=np.array(data)
while (l!=len(data)):
  l=min(n+i*n,len(data))
  y.append(data[0+i*n:l])
  i+=1



In [147]:
#LPC pour chaque trame

#Calcule des coefficients R(p)
def R(j,p):   #j représente le numéro de la trame, cette fonction calcule le coeff R(p) de la j ème trame
  r=0
  N=len(y[j])-p
  for i in range(N):
    r+=y[j][i]*y[j][i+p]
  return r

def LPC (j,P):       #Résoudre A*a=r, j représente le numéro de la trame
  r=np.zeros((P, 1))
  for i in range (P):
    r[i][0]=R(j,i+1)
  
  A=np.zeros((P, P))
  for i in range (P):
    #Valeurs sur la diagonale
    A[i][i] = R(j,0)
  #Valeurs au dessus de la diagonale 
  for i in range (P):
    for k in range (P):
      while(i+k<P):
        #A[0+k][i+k] = A[0][i]
        A[0+k][i+k] = R(j,i)       #Problème Calcule BCP TROP LENT !!!!!!!!!!
  #Ensuite pour toute valeur de la matrice en dessous de la diagonale faire = à son symétrique
  for i in range (P):
    for k in range (P):
      while(i+k<P):
        A[i+k][k] = A[k][i+k]
  return A